In [1]:
try:

    import sys
    from sys import exit
    import os
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    from matplotlib.ticker import MaxNLocator
    import datetime
    import tables 
    plt.style.use('seaborn')
    
    cwd = os.getcwd()
    sys.path.append('..\\..\\source_system')
    from indicators import *
    import visualizer as viz
    import utility_functions as uf
    sys.path.remove('..\\..\\source_system')
    os.chdir(cwd)
    
    import random
    import statistics
    import pickle
    from statistics import mean
    import operator

except Exception as e:
    
    print(e)

def write2excel( df, filename ):
    filepath = os.path.join('..', '..',  '..', 'datastore', filename) + '.xlsx'
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    df.to_excel(writer )
    writer.save()
    

In [2]:
account_type = 'backtest'
symbol = 'EUR_USD'
start_datetime = datetime.datetime(2020,1,1,0,0,0)
end_datetime = datetime.datetime(2021,1,1,0,0,0)


In [3]:
# Read data
input_data_foldername = os.path.join( '..\\..\\..\\datastore', '_{}'.format(account_type), '{}'.format(symbol) )

data_granularity = '1H'
input_data_filename = '{}.hdf'.format(data_granularity)

df = pd.DataFrame()            
df = uf.read_hdf_to_df(input_data_foldername, input_data_filename)

df = df.loc[(df.index >= start_datetime) & (df.index <= end_datetime), :]
            
df.head()

Reading dataframe from ..\..\..\datastore\_backtest\EUR_USD\1H.hdf


,ask_o,ask_h,ask_l,ask_c,bid_o,bid_h,bid_l,bid_c,volume
date,,,,,,,,,
2020-01-01 22:00:00,1.12158,1.12210,1.12158,1.12171,1.12084,1.12141,1.12075,1.12132,65
2020-01-01 23:00:00,1.12164,1.12258,1.12164,1.12196,1.12134,1.12210,1.12134,1.12183,462
2020-01-02 00:00:00,1.12198,1.12199,1.12164,1.12191,1.12184,1.12186,1.12151,1.12178,451
2020-01-02 01:00:00,1.12190,1.12253,1.12187,1.12217,1.12177,1.12239,1.12175,1.12203,912
2020-01-02 02:00:00,1.12216,1.12254,1.12191,1.12230,1.12202,1.12240,1.12178,1.12217,603


In [4]:
# Use only bid data, eliminate ask data
df.drop(['ask_o', 'ask_h', 'ask_l', 'ask_c'], axis=1, inplace=True)

for lag in range(1,13):
    df['log_return_{}'.format(lag)] = np.log( df['bid_c'] / df['bid_c'].shift(lag) ) 

df.dropna(inplace=True)

cols = []
cols = list(df.columns.values)

def f(row,lag):
    if row['log_return_{}'.format(lag)] >= 0.0:
        val = 1
    else:
        val = 0
    return val

for lag in range(1,13):
    df['Up_{}'.format(lag)] = df.apply(f, args=(lag,), axis=1)
    
df.tail()

,bid_o,bid_h,bid_l,bid_c,volume,log_return_1,log_return_2,log_return_3,log_return_4,log_return_5,...,Up_3,Up_4,Up_5,Up_6,Up_7,Up_8,Up_9,Up_10,Up_11,Up_12
date,,,,,,,,,,,,,,,,,,,,,
2020-12-27 22:00:00,1.21842,1.21873,1.21842,1.21850,40,0.000181,0.000172,0.000460,0.000419,0.000213,...,1,1,1,1,1,0,0,0,0,0
2020-12-27 23:00:00,1.21845,1.22063,1.21832,1.22032,1148,0.001493,0.001673,0.001665,0.001952,0.001911,...,1,1,1,1,1,1,1,1,1,1
2020-12-28 00:00:00,1.22034,1.22074,1.21803,1.21917,2999,-0.000943,0.000550,0.000730,0.000722,0.001009,...,1,1,1,1,1,1,1,1,1,1
2020-12-28 01:00:00,1.21918,1.22058,1.21902,1.22043,2345,0.001033,0.000090,0.001583,0.001763,0.001755,...,1,1,1,1,1,1,1,1,1,1
2020-12-28 02:00:00,1.22044,1.22131,1.22037,1.22104,960,0.000500,0.001533,0.000590,0.002082,0.002263,...,1,1,1,1,1,1,1,1,1,1


In [5]:
# Add_indicators

df, cols = AddWave(df, cols, 'bid', 34)
df, cols = AddWaveAngle(df, cols, 3)
df, cols = AddMACD(df, cols, 'bid', fastperiod=12, slowperiod=26, signalperiod=9)
df, cols = AddSlowStochastic(df, cols, 'bid', fastk_period=14, slowk_period=3, slowd_period=3)

df.dropna(inplace=True)

df.tail()

,bid_o,bid_h,bid_l,bid_c,volume,log_return_1,log_return_2,log_return_3,log_return_4,log_return_5,...,Up_12,waveclose,wavehigh,wavelow,waveangle,macd,macdsignal,macdhist,slowk,slowd
date,,,,,,,,,,,,,,,,,,,,,
2020-12-27 22:00:00,1.21842,1.21873,1.21842,1.21850,40,0.000181,0.000172,0.000460,0.000419,0.000213,...,0,1.218992,1.219599,1.218480,-0.107173,-0.000336,-0.000236,-0.000100,26.049161,21.376277
2020-12-27 23:00:00,1.21845,1.22063,1.21832,1.22032,1148,0.001493,0.001673,0.001665,0.001952,0.001911,...,1,1.219068,1.219657,1.218471,-0.028077,-0.000170,-0.000222,0.000053,57.892130,39.634203
2020-12-28 00:00:00,1.22034,1.22074,1.21803,1.21917,2999,-0.000943,0.000550,0.000730,0.000722,0.001009,...,1,1.219074,1.219719,1.218446,0.022432,-0.000129,-0.000204,0.000075,53.866193,46.750198
2020-12-28 01:00:00,1.21918,1.22058,1.21902,1.22043,2345,0.001033,0.000090,0.001583,0.001763,0.001755,...,1,1.219152,1.219769,1.218478,0.089582,0.000004,-0.000162,0.000167,71.981020,59.365609
2020-12-28 02:00:00,1.22044,1.22131,1.22037,1.22104,960,0.000500,0.001533,0.000590,0.002082,0.002263,...,1,1.219259,1.219857,1.218587,0.138354,0.000158,-0.000098,0.000256,82.341861,70.853735


In [6]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

score_dict = {}
for lag in range(1,13):

    X = df[cols].values
    y = df['Up_{}'.format(lag)].values

    lr = LogisticRegression()
    
    scores = cross_val_score(lr, X, y, cv=10, scoring='accuracy')
        
    score_dict[lag] = scores.mean()


C:\Users\boratarhan\.conda\envs\py375\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\boratarhan\.conda\envs\py375\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\boratar

DatetimeIndex(['2020-01-05 22:00:00', '2020-01-05 23:00:00',
               '2020-01-06 00:00:00', '2020-01-06 01:00:00',
               '2020-01-06 02:00:00', '2020-01-06 03:00:00',
               '2020-01-06 04:00:00', '2020-01-06 05:00:00',
               '2020-01-06 06:00:00', '2020-01-06 07:00:00',
               ...
               '2020-12-24 17:00:00', '2020-12-24 18:00:00',
               '2020-12-24 19:00:00', '2020-12-24 20:00:00',
               '2020-12-24 21:00:00', '2020-12-27 22:00:00',
               '2020-12-27 23:00:00', '2020-12-28 00:00:00',
               '2020-12-28 01:00:00', '2020-12-28 02:00:00'],
              dtype='datetime64[ns]', name='date', length=6100, freq=None)